In [1]:
# from qiskit.circuit.library import EfficientSU2
# from qiskit import QuantumCircuit
# from qiskit.circuit.parameter import Parameter

# import warnings


# warnings.filterwarnings("ignore", category=DeprecationWarning)

# def qrc_circuit(num_qubits):
#     phicirc = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=True,
#                     entanglement="full", reps=1, parameter_prefix="phi")
#     vcirc1 = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=False,
#                     entanglement="full", reps=1, parameter_prefix="v1")
#     vcirc2 = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=False,
#                     entanglement="full", reps=1, parameter_prefix="v2")
#     qc = QuantumCircuit(num_qubits, num_qubits)
#     qc = qc.compose(phicirc)
#     qc = qc.compose(vcirc1)
#     qc = qc.compose(vcirc2)
#     f_b = Parameter("f_b b x")
#     zs = [Parameter(f"b y_{i}") for i in range(num_qubits)]
#     vars = [f_b]+zs[:-1]+ zs*4
#     qrc = qc.assign_parameters(vars)
#     return qrc

# num_qubits = 3
# circuit = qrc_circuit(num_qubits)
# print("Qiskit QRC Circuit:")
# print(circuit.decompose().draw(fold=-1))


In [2]:
import cudaq
import numpy as np
from typing import List

# Helper function to apply rotation layer
@cudaq.kernel
def apply_rotations(qubits: cudaq.qview, params: List[float], start_idx: int):
    """Apply RY rotations to all qubits"""
    for i in range(len(qubits)):
        ry(params[start_idx + i], qubits[i])

# Helper function to apply full entanglement
@cudaq.kernel  
def apply_full_entanglement(qubits: cudaq.qview):
    """Apply CNOT gates between all pairs (full entanglement)"""
    num_qubits = len(qubits)
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])

@cudaq.kernel
def qrc_circuit_cudaq(num_qubits: int, params: List[float]):
    """
    CUDA-Q implementation of the QRC circuit from Qiskit
    - phicirc: EfficientSU2 with skip_final_rotation_layer=True
    - vcirc1: EfficientSU2 with skip_final_rotation_layer=False  
    - vcirc2: EfficientSU2 with skip_final_rotation_layer=False
    """
    qubits = cudaq.qvector(num_qubits)
    
    # Track parameter index
    param_idx = 0
    
    # === PHI CIRCUIT (skip_final_rotation=True) ===
    # First rotation layer
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    
    # Full entanglement
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    
    # No final rotation (skip_final_rotation=True)
    
    # === V1 CIRCUIT (skip_final_rotation=False) ===
    # First rotation layer
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    
    # Full entanglement
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    
    # Final rotation layer
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    
    # === V2 CIRCUIT (skip_final_rotation=False) ===
    # First rotation layer
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1
    
    # Full entanglement
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            x.ctrl(qubits[i], qubits[j])
    
    # Final rotation layer
    for i in range(num_qubits):
        ry(params[param_idx], qubits[i])
        param_idx += 1


# Test the circuit
num_qubits = 3

# Calculate total parameters needed:
# phicirc: num_qubits params (skip final rotation)
# vcirc1: 2*num_qubits params (rotation + entanglement + rotation)
# vcirc2: 2*num_qubits params (rotation + entanglement + rotation)
total_params = num_qubits + 2*num_qubits + 2*num_qubits

print(f"Number of qubits: {num_qubits}")
print(f"Total parameters needed: {total_params}")

# Initialize with random parameters
np.random.seed(42)
params = np.random.uniform(0, 2*np.pi, total_params).tolist()

print("\n" + "="*60)
print("CUDA-Q QRC Circuit:")
print("="*60)
print(cudaq.draw(qrc_circuit_cudaq, num_qubits, params))

# Test sampling
print("\n" + "="*60)
print("Sampling the circuit:")
print("="*60)
result = cudaq.sample(qrc_circuit_cudaq, num_qubits, params, shots_count=1000)
print(result)



Number of qubits: 3
Total parameters needed: 15

CUDA-Q QRC Circuit:
     ╭───────────╮          ╭───────────╮                        ╭───────────╮»
q0 : ┤ ry(2.353) ├──●────●──┤ ry(3.761) ├────────────────●────●──┤ ry(0.365) ├»
     ├───────────┤╭─┴─╮  │  ╰───────────╯╭────────────╮╭─┴─╮  │  ╰───────────╯»
q1 : ┤ ry(5.974) ├┤ x ├──┼────────●──────┤ ry(0.9803) ├┤ x ├──┼────────●──────»
     ├───────────┤╰───╯╭─┴─╮    ╭─┴─╮    ├────────────┤╰───╯╭─┴─╮    ╭─┴─╮    »
q2 : ┤ ry(4.599) ├─────┤ x ├────┤ x ├────┤ ry(0.9801) ├─────┤ x ├────┤ x ├────»
     ╰───────────╯     ╰───╯    ╰───╯    ╰────────────╯     ╰───╯    ╰───╯    »

################################################################################

╭───────────╮                        ╭──────────╮             
┤ ry(4.449) ├────────────────●────●──┤ ry(5.23) ├─────────────
├───────────┤╭────────────╮╭─┴─╮  │  ╰──────────╯╭───────────╮
┤ ry(5.442) ├┤ ry(0.1293) ├┤ x ├──┼───────●──────┤ ry(1.334) ├
├───────────┤├───────────┬╯╰───╯╭─┴─

In [3]:
available_backends = cudaq.get_targets()
for backend in available_backends:
    print(f"• {backend}")
print()


• Target tensornet-mps
	simulator=tensornet_mps
	platform=default
	description=cutensornet simulator backend target based on matrix product state representation
	precision=fp64
Supported Arguments:
  - option (Specify the target options as a comma-separated list.
Supported options are 'fp32', 'fp64')

• Target stim
	simulator=stim
	platform=default
	description=Stim-based CPU-only backend target
	precision=fp64

• Target quera
	simulator=qpp
	platform=default
	description=CUDA-Q target for QuEra.
	precision=fp32
Supported Arguments:
  - machine (Specify the QuEra QPU.)
  - default_bucket (Specify a default S3 bucket for QuEra results.)

• Target quantum_machines
	simulator=qpp
	platform=default
	description=CUDA-Q target for Quantum Machines.
	precision=fp32
Supported Arguments:
  - url (Specify Quantum Machine base server url.)
  - executor (Specify the executor to run. Default is mock)
  - api_key (An API key to access the Qoperator server)

• Target pasqal
	simulator=qpp
	platform=d

In [5]:
# Set the target once - use qpp-cpu (stable default) or tensornet
# DO NOT switch targets multiple times - it can crash the kernel
cudaq.set_target("qpp-cpu")

# Run the circuit with sampling
num_shots = 1000
result = cudaq.sample(qrc_circuit_cudaq, num_qubits, params, shots_count=num_shots)

print(f"\nSampling results ({num_shots} shots):")
print(result)

# Get the statevector (uses same target as above)
state_result = cudaq.get_state(qrc_circuit_cudaq, num_qubits, params)
state_array= np.array(state_result)
print(f"\nStatevector shape: {state_array.shape}")
print(f"First few amplitudes: {state_array[:4]}")
print(f"Statevector type: {type(state_result)}")
print(f"Array type: {type(state_array)}")



Sampling results (1000 shots):
{ 000:4 001:38 010:24 011:287 100:50 101:7 111:590 }


Statevector shape: (8,)
First few amplitudes: [0.05488205+0.j 0.23584448+0.j 0.15176922+0.j 0.01587004+0.j]
Statevector type: <class 'cudaq.mlir._mlir_libs._quakeDialects.cudaq_runtime.State'>
Array type: <class 'numpy.ndarray'>


In [6]:
class QuantumReservoirCUDAQ:
    """
    CUDA-Q implementation of Quantum Reservoir Computing.
    """
    
    def __init__(self, num_qubits, target="qpp-cpu"):
        self.num_qubits = num_qubits
        self.total_params = num_qubits + 2*num_qubits + 2*num_qubits  # 5*num_qubits
        cudaq.set_target(target)
        
    def efficient_su2_kernel(self, qubits, params, skip_final=False):
        """Helper to build EfficientSU2 layer."""
        n = len(qubits)
        idx = 0
        
        # Rotation layer
        for i in range(n):
            cudaq.ry(params[idx], qubits[i])
            idx += 1
        
        # Full entanglement
        for i in range(n):
            for j in range(i + 1, n):
                cudaq.cx(qubits[i], qubits[j])
        
        # Second rotation (if not skipped)
        if not skip_final:
            for i in range(n):
                cudaq.ry(params[idx], qubits[i])
                idx += 1
        
        return idx
    
    def sample(self, params, shots=1000):
        """Execute the circuit and return measurement results."""
        result = cudaq.sample(qrc_circuit_cudaq, self.num_qubits, params, shots_count=shots)
        return result
    
    def get_expectation_z(self, params, qubit_idx=0):
        """Get expectation value of Z on a specific qubit."""
        @cudaq.kernel
        def measure_z(num_qubits: int, params: list[float], target_qubit: int):
            qubits = cudaq.qvector(num_qubits)
            
            param_idx = 0
            
            # phicirc
            phi_params = params[param_idx:param_idx + num_qubits]
            for i in range(num_qubits):
                ry(phi_params[i], qubits[i])
            param_idx += num_qubits
            
            for i in range(num_qubits):
                for j in range(i + 1, num_qubits):
                    cx(qubits[i], qubits[j])
            
            # vcirc1
            v1_params = params[param_idx:param_idx + 2*num_qubits]
            for i in range(num_qubits):
                ry(v1_params[i], qubits[i])
            param_idx += num_qubits
            
            for i in range(num_qubits):
                for j in range(i + 1, num_qubits):
                    cx(qubits[i], qubits[j])
            
            for i in range(num_qubits):
                ry(v1_params[num_qubits + i], qubits[i])
            param_idx += num_qubits
            
            # vcirc2
            v2_params = params[param_idx:param_idx + 2*num_qubits]
            for i in range(num_qubits):
                ry(v2_params[i], qubits[i])
            param_idx += num_qubits
            
            for i in range(num_qubits):
                for j in range(i + 1, num_qubits):
                    cx(qubits[i], qubits[j])
            
            for i in range(num_qubits):
                ry(v2_params[num_qubits + i], qubits[i])
        
        # Observe Z on target qubit
        from cudaq import spin
        hamiltonian = spin.z(qubit_idx)
        exp_val = cudaq.observe(measure_z, hamiltonian, self.num_qubits, params, qubit_idx)
        return exp_val.expectation()


# Example usage
qrc = QuantumReservoirCUDAQ(num_qubits=3)

# Sample with random parameters
print("\\n--- CUDA-Q Quantum Reservoir Computing ---")
result = qrc.sample(params, shots=1000)
print(f"Measurement results:\\n{result}")

# Get expectation values (useful for readout in reservoir computing)
exp_z0 = qrc.get_expectation_z(params, qubit_idx=0)
exp_z1 = qrc.get_expectation_z(params, qubit_idx=1)
exp_z2 = qrc.get_expectation_z(params, qubit_idx=2)

print(f"\\nExpectation values (readout):")
print(f"  <Z_0> = {exp_z0:.4f}")
print(f"  <Z_1> = {exp_z1:.4f}")
print(f"  <Z_2> = {exp_z2:.4f}")


\n--- CUDA-Q Quantum Reservoir Computing ---
Measurement results:\n{ 000:2 001:33 010:24 011:311 100:49 101:13 111:568 }

\nExpectation values (readout):
  <Z_0> = -0.2685
  <Z_1> = -0.7983
  <Z_2> = -0.8362
